# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nemuro,43.3236,145.5750,14.96,82,5,2.72,JP,1724182883
1,1,tiksi,71.6872,128.8694,3.55,93,100,3.30,RU,1724182884
2,2,enewetak,11.3474,162.3373,28.33,79,81,6.42,MH,1724182886
3,3,adamstown,-25.0660,-130.1015,17.79,58,99,8.18,PN,1724182888
4,4,hermanus,-34.4187,19.2345,10.45,84,0,4.47,ZA,1724182889


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
first_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True,      
    tiles='OSM',
    color='City',
    size='Humidity',  
    alpha = .7,
    frame_width=700, frame_height=500, 
    
)

# Display the map
first_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city = city_data_df[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27)]  
filtered_wind = filtered_city[(filtered_city['Wind Speed'] < 5)] 
filtered_cloud = filtered_wind[(filtered_wind['Cloudiness'] <= 15)] 
# Drop any rows with null values
cleaned_filtered_df = filtered_cloud.dropna()

# Display sample data
cleaned_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,hell-ville,-13.4000,48.2667,25.34,63,6,3.05,MG,1724182918
52,52,weno,7.4515,151.8468,26.97,89,0,1.54,FM,1724182945
74,74,razdol'noye,45.7708,33.4878,24.25,63,0,3.44,UA,1724182971
97,97,argo,37.6333,22.7333,26.89,55,10,2.59,GR,1724183000
108,108,antonio enes,-16.2325,39.9086,23.32,75,4,3.80,MZ,1724183012
116,116,miranda do douro,41.4969,-6.2731,25.71,35,0,3.85,PT,1724183021
120,120,east london,-33.0153,27.9116,22.79,34,6,3.84,ZA,1724183025
150,150,crikvenica,45.1772,14.6928,26.11,61,0,2.06,HR,1724183060
226,226,international falls,48.6011,-93.4110,25.08,60,0,4.12,US,1724183146
245,245,vinhais,41.8351,-7.0050,21.21,52,0,2.80,PT,1724183171


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
copy_df = cleaned_filtered_df.copy() 

hotel_df = copy_df[['City','Country','Lat','Lng','Humidity']] 

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = np.nan

hotel_df = hotel_df.reset_index(drop=True)
# Display sample data
hotel_df

C:\Users\18056\AppData\Local\Temp\ipykernel_2184\580372599.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = np.nan


,City,Country,Lat,Lng,Humidity,Hotel Name
0,hell-ville,MG,-13.4000,48.2667,63,NaN
1,weno,FM,7.4515,151.8468,89,NaN
2,razdol'noye,UA,45.7708,33.4878,63,NaN
3,argo,GR,37.6333,22.7333,55,NaN
4,antonio enes,MZ,-16.2325,39.9086,75,NaN
5,miranda do douro,PT,41.4969,-6.2731,35,NaN
6,east london,ZA,-33.0153,27.9116,34,NaN
7,crikvenica,HR,45.1772,14.6928,61,NaN
8,international falls,US,48.6011,-93.4110,60,NaN
9,vinhais,PT,41.8351,-7.0050,52,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "apiKey" : geoapify_key ,
    "limit" : 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"] 

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}" 
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


C:\Users\18056\AppData\Local\Temp\ipykernel_2184\3258684393.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'ABUD HOTEL' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]


hell-ville - nearest hotel: ABUD HOTEL
weno - nearest hotel: High Tide Hotel
razdol'noye - nearest hotel: Портовое
argo - nearest hotel: Apollon Hotel
antonio enes - nearest hotel: Hotel Quirimbas
miranda do douro - nearest hotel: Hotel Parador Santa Catarina
east london - nearest hotel: No hotel found
crikvenica - nearest hotel: Ivančica
international falls - nearest hotel: Copper River Inn & Conference Centre
vinhais - nearest hotel: Vinhais Hotel
maragogi - nearest hotel: Pousada Mariluz
al abyar - nearest hotel: No hotel found
dryden - nearest hotel: Holiday Inn Express
urla - nearest hotel: No hotel found
cabinda - nearest hotel: Hotel Mayombe
rockland - nearest hotel: Cullens Hotel
pacific grove - nearest hotel: Pacific Grove Inn
shibganj - nearest hotel: No hotel found
xiaoweizhai - nearest hotel: No hotel found
waynesville - nearest hotel: Andon-Reid Inn Bed and Breakfast
nchelenge - nearest hotel: No hotel found
paraty - nearest hotel: Solar dos Gerânios
yutan - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hell-ville,MG,-13.4000,48.2667,63,ABUD HOTEL
1,weno,FM,7.4515,151.8468,89,High Tide Hotel
2,razdol'noye,UA,45.7708,33.4878,63,Портовое
3,argo,GR,37.6333,22.7333,55,Apollon Hotel
4,antonio enes,MZ,-16.2325,39.9086,75,Hotel Quirimbas
5,miranda do douro,PT,41.4969,-6.2731,35,Hotel Parador Santa Catarina
6,east london,ZA,-33.0153,27.9116,34,No hotel found
7,crikvenica,HR,45.1772,14.6928,61,Ivančica
8,international falls,US,48.6011,-93.4110,60,Copper River Inn & Conference Centre
9,vinhais,PT,41.8351,-7.0050,52,Vinhais Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,      
    tiles='OSM',
    color='City',
    size='Humidity',  
    alpha=0.7,
    hover_cols=['City', 'Hotel Name', 'Country'], 
    frame_width=700, 
    frame_height=500
)

# Display the map
hotel_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)